## data

In [1]:
# list of art related words
words = ['abstract',
 'aesthetic',
 'acrylic',
 'artistry',
 'animation',
 'brushwork',
 'canvas',
 'ceramics',
 'collage',
 'color',
 'composition',
 'creativity',
 'culture',
 'design',
 'drawing',
 'easel',
 'expression',
 'fresco',
 'gallery',
 'graffiti',
 'hue',
 'illustration',
 'impressionism',
 'ink',
 'installation',
 'landscape',
 'masterpiece',
 'medium',
 'mural',
 'museum',
 'oil',
 'palette',
 'pastel',
 'perspective',
 'photography',
 'pigment',
 'portrait',
 'realism',
 'sculpture',
 'sketch',
 'still life',
 'surrealism',
 'texture',
 'tone',
 'watercolor',
 'abstract expressionism',
 'art deco',
 'baroque',
 'byzantine',
 'carving',
 'chiaroscuro',
 'cubism',
 'dadaism',
 'etching',
 'expressionism',
 'fauvism',
 'genre',
 'gouache',
 'harmony',
 'impression',
 'juxtaposition',
 'kinetic',
 'line',
 'minimalism',
 'modernism',
 'neoclassicism',
 'ornament',
 'perspective',
 'pop art',
 'post-impressionism',
 'realism',
 'renaissance',
 'rococo',
 'romanticism',
 'satire',
 'shade',
 'silhouette',
 'symmetry',
 'tapestry',
 'tempera',
 "trompe l'oeil",
 'urban art',
 'vanguard',
 'veneer',
 'vignette',
 'whimsical',
 'xenography',
 'yield',
 'zenith',
 'zest',
 'fresco',
 'impasto',
 'montage',
 'opus',
 'palette knife',
 'quattrocento',
 'relief',
 'stipple',
 'underpainting',
 'varnish']

In [2]:
# sanity check
len(words)

100

## get embeddings

In [3]:
from transformers import AutoTokenizer, DistilBertModel
import torch

# https://huggingface.co/distilbert-base-uncased
# https://huggingface.co/docs/transformers/v4.35.0/en/model_doc/distilbert
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
# get embedding for each class
# ❗️ note: I am averaging the embeddings for each word in the class
# ❓ question: are we interested in the final contextual embedding for each class? currently, we're looking at the final hidden state.
embeddings = []
for i in range(len(words)):
    input_ids = torch.tensor(tokenizer.encode(words[i])).unsqueeze(0)
    outputs = model(input_ids)
    last_hidden_states = outputs[0]
    # skip the first token, which is the [CLS] token, and skip the last token, which is the [SEP] token
    # average the rest of the tokens
    embeddings.append(last_hidden_states[0][1:-1].mean(dim=0).tolist())

In [8]:
# sanity check
print(len(embeddings))
print(len(embeddings[0]))

100
768


In [9]:
import numpy as np
# round each val in embedding to 3 decimal places
embeddings = [list(np.around(np.array(e),3)) for e in embeddings]

# create string of all classes and their embeddings & save to text file
# ❗️ note: only taking first 10 axes for now due to context window length
with open("output.txt", "w") as text_file:
    for i in range(len(words)):
        class_str = f"{words[i]}: {embeddings[i][:10]}\n"
        text_file.write(class_str)

## dataframe

In [12]:
# convert embeddings to pandas dataframe
import pandas as pd
df = pd.DataFrame(embeddings)
df.insert(0, 'word', words)

# sanity check
df.head()

,word,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
0,abstract,0.286,0.395,-0.382,-0.242,0.407,0.010,-0.199,0.060,0.203,...,0.734,-0.119,0.476,0.058,0.239,-0.068,0.101,0.026,0.260,-0.127
1,aesthetic,0.249,0.566,-0.123,-0.117,0.271,0.083,0.036,-0.069,0.083,...,0.516,0.024,0.273,-0.004,0.114,-0.206,0.080,0.043,0.187,0.201
2,acrylic,0.217,0.234,-0.019,0.087,0.777,-0.107,-0.655,0.548,-0.061,...,0.254,-0.524,0.176,0.345,0.337,-0.254,-0.499,-0.021,0.162,0.142
3,artistry,0.147,0.263,-0.044,-0.078,0.660,0.147,-0.043,-0.021,-0.120,...,0.639,-0.331,0.187,0.041,-0.143,-0.029,0.110,-0.205,0.363,-0.308
4,animation,-0.006,0.449,-0.484,0.105,0.504,0.228,0.095,0.199,-0.532,...,0.550,-0.206,0.477,0.005,0.140,-0.114,0.244,-0.032,0.513,0.498


In [17]:
# normalize each column to be between -1 and 1
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(-1,1))
df.iloc[:,1:] = scaler.fit_transform(df.iloc[:,1:])

# sanity check
df.head()

,word,0,1,2,3,4,5,6,7,8,...,758,759,760,761,762,763,764,765,766,767
0,abstract,0.223359,0.486683,-0.490754,-0.152763,-0.189744,0.078056,-0.045004,-0.449067,0.400238,...,0.338893,0.285846,0.473819,0.261603,0.072235,0.110902,0.214545,0.294118,0.164329,-0.366919
1,aesthetic,0.178808,0.693705,-0.122333,0.118093,-0.364103,0.185567,0.313501,-0.634146,0.257720,...,0.008340,0.550416,-0.044700,0.130802,-0.209932,-0.148496,0.176364,0.329205,0.018036,0.183879
2,acrylic,0.140277,0.291768,0.025605,0.560130,0.284615,-0.094256,-0.740656,0.251076,0.086698,...,-0.388931,-0.463460,-0.292465,0.867089,0.293454,-0.238722,-0.876364,0.197110,-0.032064,0.084803
3,artistry,0.055990,0.326877,-0.009957,0.202600,0.134615,0.279823,0.192982,-0.565280,0.016627,...,0.194845,-0.106383,-0.264368,0.225738,-0.790068,0.184211,0.230909,-0.182663,0.370741,-0.670865
4,animation,-0.128236,0.552058,-0.635846,0.599133,-0.065385,0.399116,0.403509,-0.249641,-0.472684,...,0.059894,0.124884,0.476373,0.149789,-0.151242,0.024436,0.474545,0.174407,0.671343,0.682620


In [18]:
# save to csv
df.to_csv("output.csv", index=False)

## interpreting axes with chatgpt

I uploaded the csv generated above and used the following prompt:
```
This CSV contains a list of words and their embeddings (each column after the word represents an axis in the embedding. 

By carefully comparing and considering the embedding values for each word, please interpret the likely linguistic feature that each embedding axis encodes. This interpretation must be consistent across all the words and correspond to their respective positive, zero, or negative embedding values.  You might consider analyzing the top 10 words with values close to -1, the top 10 words with values close to 0 (median), and the top 10 words with values close to 1 to generate your interpretation. 

Please phrase your interpretation using 3 words like: negative vs positive, small vs large, etc (some contrast with "vs" should be present). You should only have one interpretation per axis.

For each axis, also include a confidence score of how confident you are in your interpretation of each axis.

For each axis, the output should look like this: {<interpretation>:<interpretation confidence score>} (e.g., {"positive vs. negative": 0.6}) Remember <interpretation> should only have 3 words, including "vs".

Let's start with the first axis. Remember to format your output as requested above as a python dictionary.
```

Then repeated to get 3 interpretations for the first 10 axes.

In [32]:
# interpretations of first 10 axes:  
# each dict item is formatted as {<interpretation>: <confidence score>}
axis_0 = {"traditional vs. modern": 0.7, "traditional vs. modern": 0.6, "technique vs. medium": 0.6}
axis_1 = {"classic vs. diverse": 0.65, "detail vs. abstract": 0.6, "classical vs. modern": 0.65}
axis_2 = {"abstract vs. concrete": 0.6, "complex vs. simple": 0.6, "abstract vs. tangible": 0.6}
axis_3 = {"formal vs. playful": 0.65, "static vs. dynamic": 0.6, "formal vs. playful": 0.65}
axis_4 = {"serious vs. vibrant": 0.65, "concrete vs. expressive": 0.6, "realistic vs. expressive": 0.6}
axis_5 = {"innovative vs. traditional": 0.7, "abstract vs. concrete": 0.6, "avant-garde vs. conventional": 0.65}
axis_6 = {"material vs. conceptual": 0.65, "playful vs. serious": 0.6, "traditional vs. modern": 0.65}
axis_7 = {"modern vs. historical": 0.7, "modern vs. classical": 0.6, "contemporary vs. historical": 0.65}
axis_8 = {"innovative vs. fundamental": 0.65, "visual vs. conceptual": 0.6, "innovative vs. foundational": 0.65}
axis_9 = {"modern vs. classical": 0.7, "innovative vs. traditional": 0.6, "modern vs. classical": 0.65}

# concatenate all axes into one list
all_axes = [axis_0, axis_1, axis_2, axis_3, axis_4, axis_5, axis_6, axis_7, axis_8, axis_9]

In [33]:
# compute descriptive stats for each axis' confidence scores
# store in dict where key is axis number and value is dict of descriptive stats
import statistics

axes_stats = {}
for i in range(len(all_axes)):
    axis = all_axes[i]
    mean = statistics.mean(axis.values())
    median = statistics.median(axis.values())
    stdev = statistics.stdev(axis.values())

    # round each val to 3 decimal places
    mean = round(mean, 3)
    median = round(median, 3)
    stdev = round(stdev, 3)

    # store in dict
    axes_stats[i] = {"mean": mean, "median": median, "stdev": stdev}

print(axes_stats)

{0: {'mean': 0.6, 'median': 0.6, 'stdev': 0.0}, 1: {'mean': 0.633, 'median': 0.65, 'stdev': 0.029}, 2: {'mean': 0.6, 'median': 0.6, 'stdev': 0.0}, 3: {'mean': 0.625, 'median': 0.625, 'stdev': 0.035}, 4: {'mean': 0.617, 'median': 0.6, 'stdev': 0.029}, 5: {'mean': 0.65, 'median': 0.65, 'stdev': 0.05}, 6: {'mean': 0.633, 'median': 0.65, 'stdev': 0.029}, 7: {'mean': 0.65, 'median': 0.65, 'stdev': 0.05}, 8: {'mean': 0.633, 'median': 0.65, 'stdev': 0.029}, 9: {'mean': 0.625, 'median': 0.625, 'stdev': 0.035}}


In [34]:
# overall mean, median, and stdev of all axes' confidence scores
mean = statistics.mean([statistics.mean(axis.values()) for axis in all_axes])
median = statistics.median([statistics.median(axis.values()) for axis in all_axes])
stdev = statistics.stdev([statistics.stdev(axis.values()) for axis in all_axes])

# print overall mean, median, and stdev
print(f"mean: {mean}")
print(f"median: {median}")
print(f"stdev: {stdev}")

mean: 0.6266666666666667
median: 0.6375
stdev: 0.017137976669804756


Notes:
- decent mean confidence
- low stdev

## analyzing interpretations

I then asked chatgpt to assess how similar the 3 interpretations were for each axis and create a summary interpretation. Again, I asked it to do this 3 times to see how reliable the results were.

```
Below I will provide interpretations and confidence scores for axes in high dimensional word embeddings. Each axis has 3 potential interpretations + corresponding confidence scores. For each axis, please assign a qualitative similarity rating from 1-10 to how similar the interpretations are (1: not at all similar, 10: identical). Then summarize the three interpretations into a single interpretation per axis by considering the confidence scores, similarities between the interpretations, the most common interpretations, etc. This summary interpretation does not have to be one of the original three interpretations word for word, but it can be. Keep the words in the same relative order, as the first word in the interpretation represents what negative embedding values stand for in the axis, while the second word after vs. represents what positive embedding values stand for.

Your final answer for each axis should be a python dict formatted as follows: {<interpretation>: <similarity rating>}. As before, each interpretation should consist of exactly 3 words (including "vs"). And the similarity rating should just be the number from 1-10.

Here are the axis interpretations:
# insert here

Remember to format your answer for each axis as a python dict as requested above. You should not need to write code do this, please just do this qualitatively. Also make sure there's no duplicate interpretations.
```

In [47]:
# final axes dict
axis_0 = {"traditional vs. modern": [4,8,7]}
axis_1 = {"classic vs. modern": [5,6,4]}
axis_2 = {"abstract vs. concrete": [8,9,8]}
axis_3 = {"formal vs. playful": [7,9,8]}
axis_4 = {"serious vs. expressive": [6,7,5]}
axis_5 = {"innovative vs. traditional": [7,9,6]}
axis_6 = {"traditional vs. conceptual": [5,5,4]}
axis_7 = {"modern vs. historical": [9,9,9]}
axis_8 = {"innovative vs. foundational": [7,8,7]}
axis_9 = {"modern vs. traditional": [9,9,8]}

# concatenate all axes into one list
all_axes = [axis_0, axis_1, axis_2, axis_3, axis_4, axis_5, axis_6, axis_7, axis_8, axis_9]

In [48]:
# compute descriptive stats for each axis' confidence scores
# store in dict where key is axis number and value is dict of descriptive stats
axes_stats = {}
for i in range(len(all_axes)):
    axis = all_axes[i]
    # value is a list of confidence scores
    mean = statistics.mean(list(axis.values())[0])
    median = statistics.median(list(axis.values())[0])
    stdev = statistics.stdev(list(axis.values())[0])

    # round each val to 3 decimal places
    mean = round(mean, 3)
    median = round(median, 3)
    stdev = round(stdev, 3)

    # store in dict
    axes_stats[i] = {"mean": mean, "median": median, "stdev": stdev}

print(axes_stats)

# overall mean, median, and stdev of all axes' confidence scores
mean = statistics.mean([statistics.mean(list(axis.values())[0]) for axis in all_axes])
median = statistics.median([statistics.median(list(axis.values())[0]) for axis in all_axes])
stdev = statistics.stdev([statistics.stdev(list(axis.values())[0]) for axis in all_axes])

# print overall mean, median, and stdev
print(f"mean: {mean}")
print(f"median: {median}")
print(f"stdev: {stdev}")

{0: {'mean': 6.333, 'median': 7, 'stdev': 2.082}, 1: {'mean': 5, 'median': 5, 'stdev': 1.0}, 2: {'mean': 8.333, 'median': 8, 'stdev': 0.577}, 3: {'mean': 8, 'median': 8, 'stdev': 1.0}, 4: {'mean': 6, 'median': 6, 'stdev': 1.0}, 5: {'mean': 7.333, 'median': 7, 'stdev': 1.528}, 6: {'mean': 4.667, 'median': 5, 'stdev': 0.577}, 7: {'mean': 9, 'median': 9, 'stdev': 0.0}, 8: {'mean': 7.333, 'median': 7, 'stdev': 0.577}, 9: {'mean': 8.667, 'median': 9, 'stdev': 0.577}}
mean: 7.066666666666666
median: 7.0
stdev: 0.581747475245294


Notes:
- decently high mean similarity rating
- stdev seems reasonable

## todos
- generate synthetic embeddings using interpretations
- compare to original